In [1]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    WELFAKE
'''



data = pd.read_csv(r"../data/Des_fake_news/WELFAKE_PROCESSED.csv")
data["flag"] = data["label"]
data = data.sample(10)
data



,Unnamed: 0.1,Unnamed: 0,title,text,label,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub,title_vader_comp,title_vader_neg,title_vader_neu,title_vader_pos,text_vader_neg,text_vader_neu,text_vader_pos,flag
4967,18775,18775,Russia Brags About Helping Trump Win As Our E...,Russia and Vladimir Putin got what they wanted...,1,0.095177,0.379331,0.400000,0.200000,0.1280,0.256,0.465,0.279,0.100,0.787,0.113,1
401,48322,48322,As Syria war tightens U S and Russia militar...,AL UDEID AIR BASE Qatar Reuters Even as t...,0,-0.000174,0.291768,-0.100000,0.100000,-0.5994,0.281,0.719,0.000,0.066,0.899,0.036,0
2710,58829,58829,Debunked The Photo Of Obama With A Speech Bal...,Email \nIn an age where information can be spr...,1,0.187857,0.546633,0.000000,0.000000,0.0000,0.000,1.000,0.000,0.052,0.801,0.148,1
2346,67109,67109,Top US Spy Agency Refuses to Endorse CIAs Russ...,st Century Wire says If anyone still thinks th...,1,0.059429,0.501429,0.125000,0.291667,0.2023,0.119,0.670,0.211,0.113,0.784,0.102,1
1176,32457,32457,Alec Baldwin Joins Anti Trump Protest Gives ...,Over the next few days America and the world...,1,0.027786,0.394889,0.000000,0.000000,-0.8331,0.424,0.576,0.000,0.098,0.814,0.088,1
100,25563,25563,Behind anti Trump protests worries that Ameri...,For many Americans who have long felt threaten...,0,0.066157,0.441716,-0.400000,0.700000,-0.5719,0.405,0.462,0.133,0.101,0.800,0.099,0
90,1974,1974,American Express disowns Pink Floyd singer Rog...,American Express disowns Pink Floyd singer Rog...,1,0.103821,0.308635,-0.050000,0.150000,0.0000,0.000,1.000,0.000,0.071,0.865,0.064,1
2047,71792,71792,Justice Department antitrust nominee Makan Del...,WASHINGTON Reuters The confirmation testim...,0,-0.002101,0.236975,0.000000,0.000000,0.3612,0.154,0.569,0.276,0.067,0.863,0.070,0
2533,31581,31581,Who Will Tell the Story of Slavery The New ...,RICHMOND VA Here in the onetime capital ...,0,0.039572,0.376275,0.136364,0.454545,-0.7003,0.324,0.676,0.000,0.083,0.830,0.087,0
317,19325,19325,The Circus of Liars America s Three Rings of...,Wed Oct UTC Jen Psaki President Obama ho...,1,0.051658,0.472647,-1.000000,1.000000,-0.8316,0.464,0.536,0.000,0.140,0.765,0.095,1


In [2]:
'''
    Add tf-idf vectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd


def get_tf_idf_vector(x: pd.DataFrame, corpus: pd.Series) -> tuple[pd.DataFrame, list[str], TfidfVectorizer]:
    #temp = pd.concat((x, corpus), axis=1).reset_index()

    stop_words = stopwords.words("english")
    vectorizer = TfidfVectorizer(strip_accents="ascii", lowercase=True, stop_words=stop_words, max_features=500, ngram_range=(1,3))
    tf_idf_features = vectorizer.fit_transform(corpus).toarray()

    names = vectorizer.get_feature_names_out()
    headers = [f"__word{i}" for i in range(len(names))]
    feature_frame = pd.DataFrame(tf_idf_features, columns=headers).reset_index()
    #all_features = pd.concat((x.reset_index(), feature_frame), axis=1).reset_index()
    return (feature_frame.drop(["level_0", "index"], axis=1), names, vectorizer)



In [3]:
from openpyxl import load_workbook, Workbook

'''
    Define filtered dataset, classes, features, dataframe for model accuracies, and excel file for results
'''

'''Drop NA'''
filtered = data.dropna()
classes = filtered["flag"]

filtered["corpus"] = filtered.apply(lambda x: " ".join([x["title"], x["text"]]), axis=1)


# raw polarity and subjectivity scores from Textblob, Vader
features = filtered[["text_tb_pol",	"text_tb_sub",	"title_tb_pol",	"title_tb_sub",	"title_vader_comp",	"title_vader_neg",	
              "title_vader_neu",	"title_vader_pos",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]]



EXCEL_FILE = r"../data/Des_fake_news/Sentiment_Analysis_Results/WELFAKE_RESULTS.xlsx"
# overwrite book if exists
book = Workbook()
book.save(filename=EXCEL_FILE)
book.close()


In [ ]:
'''
    POLARITY + TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io



# average accuracies
comb_output = {"train" : None, "test" : None}
comb_matrices = []


comb_fpr = []
comb_tpr = []
comb_thresh = []

comb_auc = 0


train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

x_train = None
x_test = None
for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    tf_idf_training, names, vectorizer = get_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index])
    x_train = scaler.fit_transform(x_train)
    

    print("tf_idf testing")
    x_test = pd.DataFrame(vectorizer.transform(filtered["corpus"].iloc[test_index]).toarray(), columns=[f"__word{i}" for i in range(len(names))])
    x_test = pd.concat((x.iloc[test_index].reset_index(), x_test), axis=1).drop("index", axis=1)
    x_test = scaler.transform(x_test)
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    comb_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    comb_ensemble.fit(x_train, y_train)
    y_pred = comb_ensemble.predict(x_test)

    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    comb_fpr.append(fpr)
    comb_tpr.append(tpr)
    comb_thresh.append(thresh)
    comb_auc += metrics.auc(fpr, tpr)

    comb_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = comb_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

comb_output["test"] = [(test_total / run_count)]
comb_output["train"] = [(train_total / run_count)]
comb_output = pd.DataFrame(comb_output)

print("Average confusion matrix:")
comb_matrices = np.mean(np.array(comb_matrices), axis=0)
print(comb_matrices)


comb_fpr = np.mean(np.array(comb_fpr), axis=0)
comb_tpr = np.mean(np.array(comb_tpr), axis=0)
comb_thresh = np.mean(np.array(comb_thresh), axis=0)

comb_auc = comb_auc / float(run_count)











#x_test

comb_output


0
tf_idf training
tf_idf testing
ensemble fitting
1
tf_idf training
tf_idf testing
ensemble fitting
2
tf_idf training
tf_idf testing
ensemble fitting
3
tf_idf training
tf_idf testing
ensemble fitting
4
tf_idf training
tf_idf testing
ensemble fitting
Train accuracy on 5-fold cross-validation: 0.99595
Test accuracy on 5-fold cross-validation: 0.922
Average confusion matrix:
[[443.2  42.6]
 [ 35.4 478.8]]


,train,test
0,0.99595,0.922


In [ ]:
'''
    POLARITY
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
pol_output = {"train" : None, "test" : None}
pol_matrices = []


pol_fpr = []
pol_tpr = []
pol_thresh = []

pol_auc = 0

train_total = 0
test_total = 0
run_count = 0


#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
    scaler = StandardScaler()
    print("tf_idf training")
    x_train, names, vectorizer = add_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index])
    x_train = scaler.fit_transform(x_train)
    

    print("tf_idf testing")
    x_test = pd.DataFrame(vectorizer.transform(filtered["corpus"].iloc[test_index]).toarray(), columns=[f"__word{i}" for i in range(len(names))])
    x_test = pd.concat((x.iloc[test_index].reset_index(), x_test), axis=1).drop("index", axis=1)
    x_test = scaler.transform(x_test)
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    pol_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    pol_ensemble.fit(x_train, y_train)
    y_pred = pol_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    pol_fpr.append(fpr)
    pol_tpr.append(tpr)
    pol_thresh.append(thresh)
    pol_auc += metrics.auc(fpr, tpr)

    pol_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = pol_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

pol_output["test"] = [(test_total / run_count)]
pol_output["train"] = [(train_total / run_count)]
pol_output = pd.DataFrame(pol_output)

print("Average confusion matrix:")
pol_matrices = np.mean(np.array(pol_matrices), axis=0)
print(pol_matrices)

pol_fpr = np.mean(np.array(pol_fpr), axis=0)
pol_tpr = np.mean(np.array(pol_tpr), axis=0)
pol_thresh = np.mean(np.array(pol_thresh), axis=0)

pol_auc = pol_auc / float(run_count)




pol_output

0
tf_idf training
tf_idf testing
ensemble fitting


In [ ]:
'''
    TF-IDF
'''
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import KFold


# average accuracies
tf_output = {"train" : None, "test" : None}
tf_matrices = []

train_total = 0
test_total = 0
run_count = 0

tf_fpr = []
tf_tpr = []
tf_thresh = []

tf_auc = 0

#lbgfs
x = features
y = classes
folds = 5
kf = KFold(n_splits=folds)

for i, (train_index, test_index) in enumerate(kf.split(x)):
    print(i)
     scaler = StandardScaler()
    print("tf_idf training")
    x_train, names, vectorizer = add_tf_idf_vector(x.iloc[train_index], filtered["corpus"].iloc[train_index])
    x_train = scaler.fit_transform(x_train)
    

    print("tf_idf testing")
    x_test = pd.DataFrame(vectorizer.transform(filtered["corpus"].iloc[test_index]).toarray(), columns=[f"__word{i}" for i in range(len(names))])
    x_test = pd.concat((x.iloc[test_index].reset_index(), x_test), axis=1).drop("index", axis=1)
    x_test = scaler.transform(x_test)
   
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    tf_ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True, random_state=42))
    ], voting='soft')
    print("ensemble fitting")
    tf_ensemble.fit(x_train, y_train)
    y_pred = tf_ensemble.predict(x_test)
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_pred, pos_label=1)
    tf_fpr.append(fpr)
    tf_tpr.append(tpr)
    tf_thresh.append(thresh)
    tf_auc += metrics.auc(fpr, tpr)


    tf_matrices.append(confusion_matrix(y_test, y_pred))
    test_total += accuracy_score(y_test, y_pred)

    y_train_pred = tf_ensemble.predict(x_train)
    train_total += accuracy_score(y_train, y_train_pred)

    run_count += 1

print(f"Train accuracy on {folds}-fold cross-validation: {train_total / float(run_count)}")
print(f"Test accuracy on {folds}-fold cross-validation: {test_total / float(run_count)}")

tf_output["test"] = [(test_total / run_count)]
tf_output["train"] = [(train_total / run_count)]
tf_output = pd.DataFrame(tf_output)

print("Average confusion matrix:")
tf_matrices = np.mean(np.array(tf_matrices), axis=0)
print(tf_matrices)

tf_fpr = np.mean(np.array(tf_fpr), axis=0)
tf_tpr = np.mean(np.array(tf_tpr), axis=0)
tf_thresh = np.mean(np.array(tf_thresh), axis=0)

tf_auc = tf_auc / float(run_count)



tf_output

In [ ]:
'''
    combine accuracies into one table
'''

final_results = pd.concat((pol_output, tf_output, comb_output), axis=0)
final_results.index = ["Polarity", "Tf-Idf", "Polarity + Tf-Idf"]
final_results

In [ ]:
'''
    Finally, save accuracy metrics to the spreadsheet
'''




book = load_workbook(EXCEL_FILE)
writer = pd.ExcelWriter(EXCEL_FILE, engine="openpyxl")
writer.book = book
final_results.to_excel(writer, sheet_name=f"accuracies")
book.save(filename=EXCEL_FILE)
book.close()



In [ ]:
import openpyxl.drawing
from sklearn import metrics
import matplotlib.pyplot as plt
import io


plt.figure()
lw = 2
plt.plot(pol_fpr, pol_tpr,
 lw=lw, label='Polarity: (%0.2f)' % pol_auc)
plt.plot(tf_fpr, tf_tpr,
 lw=lw, label='Tf-Idf: (%0.2f)' % tf_auc)
plt.plot(comb_fpr, comb_tpr,
 lw=lw, label='Polarity + Tf-Idf: (%0.2f)' % comb_auc)
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
#plt.show()


# save figure as PNG
png = io.BytesIO()
plt.savefig(png, format="png")


# write PNG to excel file
book = load_workbook(EXCEL_FILE)
ws = book.active

img = openpyxl.drawing.image.Image(png)
img.anchor = "A1"
ws.add_image(img)
book.save(filename=EXCEL_FILE)
plt.close()
book.close()

